In [2]:
import pandas as pd
from pulp import LpVariable, LpProblem, lpSum, LpMaximize, LpInteger

out = []
with open('/home/acohen/Desktop/fanduel') as f:
    for i in f:
        out.append(i.split(';'))
        
df = pd.DataFrame(out[1:], columns=out[0])

f = lambda x: ''.join([char.lower() for char in x if char.isalnum()])
df.columns = [f(c) for c in df.columns.tolist()]
df = df[['name', 'pos', 'salary', 'ptsgame']]
df.salary = df.salary.astype(int)
df.ptsgame = df.ptsgame.astype(float)
df = df[df.salary>0]

dN = {g[0]: g[1].name.tolist() for g in df.groupby('pos')}

dSal = {k: int(v) for (k, v) in zip(df.name, df.salary)}
dPts = {k: float(v) for (k, v) in zip(df.name, df.ptsgame)}
dPos = {k: v for (k, v) in zip(df.name, df.pos)}

players = list(set(df.name))
player_vars = LpVariable.dicts('plyrs', players, lowBound=0, upBound=1, cat=LpInteger)

prob = LpProblem('', LpMaximize)
prob += lpSum([dPts[i]*player_vars[i] for i in players])

prob += lpSum([player_vars[i] for i in players if i in dN['WR']]) == 3
prob += lpSum([player_vars[i] for i in players if i in dN['RB']]) == 2
prob += lpSum([player_vars[i] for i in players if i in dN['QB']]) == 1
prob += lpSum([player_vars[i] for i in players if i in dN['TE']]) == 1
prob += lpSum([player_vars[i] for i in players if i in dN['D']]) == 1
prob += lpSum([player_vars[i] for i in players if i in dN['K']]) == 1

prob += lpSum([dSal[i]*player_vars[i] for i in players]) <= 60000

prob.solve()

1

In [3]:
final = df[df.name.isin([v.name.replace('plyrs_', '').replace('_', ' ') for v in prob.variables() if v.varValue==1])]

final.sort_values(['pos', 'salary'], ascending=False)

,name,pos,salary,ptsgame
17,"Evans, Mike",WR,8500,19.00
44,"Thomas, Michael",WR,6400,13.85
150,"Royal, Eddie",WR,5100,11.05
107,"Reed, Jordan",TE,7000,13.42
4,"Gordon, Melvin",RB,8300,20.13
9,"Murray, DeMarco",RB,7400,19.20
54,"Brady, Tom",QB,8300,26.29
96,"Sturgis, Caleb",K,4600,10.63
80,Philadelphia,D,4400,11.50


In [4]:
sum(final.ptsgame), sum(final.salary)

(145.06999999999999, 60000)